In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

In [ ]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

In [1]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_qrnn.pkl'), "rb") as f:
    word_dict = pickle.load(f)

In [2]:
import pandas as pd
import os
train = pd.read_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=None, names=None)
print(train.shape, test_sample.shape)

(30000, 502) (20000, 502)


In [3]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((30000, 502), (10000, 502), (10000, 502))

In [4]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float().squeeze()
train_X = torch.from_numpy(train.drop([0], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float().squeeze()
val_X = torch.from_numpy(val.drop([0], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float().squeeze()
test_X = torch.from_numpy(test.drop([0], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10000])


In [5]:
import torch.nn as nn
import numpy as np

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, trial,vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_dim = trial.suggest_int("hidden_dim",16,256)
        embedding_dim = trial.suggest_int("embedding_dim",16,128)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [6]:
data_dir = './lstm-imdb-hyperband-trails_6/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./lstm-imdb-hyperband-trails_6/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [7]:
import time
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.optim as optim
import numpy as np

epochs = 10
vocab_size=5000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define an objective function to be minimized.
def objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    #epochs = trial.suggest_int("epochs",5,15)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
    #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
    model = LSTMClassifier(trial, vocab_size).to(device)
    trial.set_user_attr(key="best_model", value=model)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    loss_fn = torch.nn.BCELoss()
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_accuracy_epoch = []
    time_train = []
    final_val_acc = 0
    for epoch in range(epochs):
        model.train()
        start = time.time()
        total_loss = 0
        train_acc = 0
        total = 0
        correct = 0
        for batch in train_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            prediction = model(batch_X)
            loss = loss_fn(prediction, batch_y)
            loss.backward()
            optimizer.step()
            result = np.round(prediction.detach().cpu())
            total_loss += loss.data.item()
            total += batch_y.size(0)
            correct += (result == batch_y.cpu()).sum().item()
            train_acc = correct/total
        train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
        train_acc_epoch.append(np.round(train_acc*100,3))
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                prediction = model(inputs_val)
                loss = loss_fn(prediction, labels_val)
                val_loss.append(np.round(loss.item(),3))
                result = np.round(prediction.cpu())
                total += labels_val.size(0)
                correct += (result == labels_val.cpu()).sum().item()
            val_accuracy_epoch.append(np.round((correct/total)*100, 3))
            val_loss_epoch.append(np.round(np.mean(val_loss),3))
            end = time.time() - start
            final_val_acc = np.round((correct/total)*100, 3)
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
            time_train.append(np.round(end,3))
    write_to_csv(trial.number, epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
    return final_val_acc


In [8]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [9]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=6,callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 14:05:34,762] A new study created in memory with name: no-name-bae07840-4620-4efd-9f06-4b9646c6623a


Epoch: 0, BCELoss: 0.41769512213766574
Val Loss: 0.308 	Val Acc: 0.869
Epoch: 1, BCELoss: 0.27510410668949287
Val Loss: 0.292 	Val Acc: 0.880
Epoch: 2, BCELoss: 0.2502126902962724
Val Loss: 0.310 	Val Acc: 0.874
Epoch: 3, BCELoss: 0.3075947742909193
Val Loss: 0.650 	Val Acc: 0.637
Epoch: 4, BCELoss: 0.6259882539014021
Val Loss: 0.636 	Val Acc: 0.633
Epoch: 5, BCELoss: 0.6253706437349319
Val Loss: 0.636 	Val Acc: 0.631
Epoch: 6, BCELoss: 0.6232606396575768
Val Loss: 0.634 	Val Acc: 0.629
Epoch: 7, BCELoss: 0.6226270253956318
Val Loss: 0.640 	Val Acc: 0.629
Epoch: 8, BCELoss: 0.623735518703858
Val Loss: 0.639 	Val Acc: 0.629
Epoch: 9, BCELoss: 0.6222674574454625


[I 2022-05-14 14:08:26,946] Trial 0 finished with value: 63.0 and parameters: {'optimizer': 'Adam', 'lr': 0.009550602064809175, 'dropout': 0.5705425114414746, 'hidden_dim': 217, 'embedding_dim': 125}. Best is trial 0 with value: 63.0.


Val Loss: 0.637 	Val Acc: 0.630
Epoch: 0, BCELoss: 0.5164170065273841
Val Loss: 0.427 	Val Acc: 0.810
Epoch: 1, BCELoss: 0.35355854195853076
Val Loss: 0.632 	Val Acc: 0.740
Epoch: 2, BCELoss: 0.2864521169414123
Val Loss: 0.334 	Val Acc: 0.863
Epoch: 3, BCELoss: 0.25326452701042096
Val Loss: 0.299 	Val Acc: 0.878
Epoch: 4, BCELoss: 0.21164309836924075
Val Loss: 0.308 	Val Acc: 0.880
Epoch: 5, BCELoss: 0.17744684979319572
Val Loss: 0.325 	Val Acc: 0.879
Epoch: 6, BCELoss: 0.13770583737952014
Val Loss: 0.374 	Val Acc: 0.876
Epoch: 7, BCELoss: 0.1042228236158068
Val Loss: 0.411 	Val Acc: 0.876
Epoch: 8, BCELoss: 0.06828997010442739
Val Loss: 0.475 	Val Acc: 0.873
Epoch: 9, BCELoss: 0.047025494617410006


[I 2022-05-14 14:10:35,936] Trial 1 finished with value: 87.04 and parameters: {'optimizer': 'RMSprop', 'lr': 0.002058043533075509, 'dropout': 0.35916214323504625, 'hidden_dim': 151, 'embedding_dim': 71}. Best is trial 1 with value: 87.04.


...

Val Loss: 0.665 	Val Acc: 0.611
Epoch: 0, BCELoss: 0.6721845672527949
Val Loss: 0.643 	Val Acc: 0.623
Epoch: 1, BCELoss: 0.5927084389328957
Val Loss: 0.523 	Val Acc: 0.739
Epoch: 2, BCELoss: 0.5066125642259915
Val Loss: 0.499 	Val Acc: 0.760
Epoch: 3, BCELoss: 0.4425988132258256
Val Loss: 0.401 	Val Acc: 0.825
Epoch: 4, BCELoss: 0.39577228665351866
Val Loss: 0.377 	Val Acc: 0.838
Epoch: 5, BCELoss: 0.36124255329370497
Val Loss: 0.361 	Val Acc: 0.844
Epoch: 6, BCELoss: 0.34195550411939624
Val Loss: 0.354 	Val Acc: 0.846
Epoch: 7, BCELoss: 0.31293261736631395
Val Loss: 0.348 	Val Acc: 0.853
Epoch: 8, BCELoss: 0.29036299165338275
Val Loss: 0.339 	Val Acc: 0.857
Epoch: 9, BCELoss: 0.27100382807354134


[I 2022-05-14 14:17:11,474] Trial 4 finished with value: 85.68 and parameters: {'optimizer': 'SGD', 'lr': 0.15871486187790268, 'dropout': 0.582539890978334, 'hidden_dim': 181, 'embedding_dim': 90}. Best is trial 2 with value: 87.12.


Val Loss: 0.341 	Val Acc: 0.857
Epoch: 0, BCELoss: 0.692031569381555
Val Loss: 0.691 	Val Acc: 0.524
Epoch: 1, BCELoss: 0.6887138007084529
Val Loss: 0.688 	Val Acc: 0.542
Epoch: 2, BCELoss: 0.6843544314304988
Val Loss: 0.683 	Val Acc: 0.560
Epoch: 3, BCELoss: 0.6719852420687675
Val Loss: 0.662 	Val Acc: 0.599
Epoch: 4, BCELoss: 0.6441376140713692
Val Loss: 0.640 	Val Acc: 0.627
Epoch: 5, BCELoss: 0.6206267277896405
Val Loss: 0.622 	Val Acc: 0.657
Epoch: 6, BCELoss: 0.6011030146976312
Val Loss: 0.604 	Val Acc: 0.674
Epoch: 7, BCELoss: 0.5828266031543414
Val Loss: 0.587 	Val Acc: 0.692
Epoch: 8, BCELoss: 0.5651109232008458
Val Loss: 0.571 	Val Acc: 0.707
Epoch: 9, BCELoss: 0.548620665470759


[I 2022-05-14 14:18:50,564] Trial 5 finished with value: 72.17 and parameters: {'optimizer': 'Adam', 'lr': 4.943560870046532e-05, 'dropout': 0.5408280910085019, 'hidden_dim': 65, 'embedding_dim': 20}. Best is trial 2 with value: 87.12.


Val Loss: 0.556 	Val Acc: 0.722
Study statistics: 
  Number of finished trials:  6
  Number of complete trials:  6
Best trial:
  Value:  87.12
  Params: 
    optimizer: RMSprop
    lr: 0.003846672585823485
    dropout: 0.3015654025341833
    hidden_dim: 215
    embedding_dim: 112


In [10]:
data_dir = './lstm-imdb-hyperband/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)
    
torch.save(best_model.state_dict(),"./lstm-imdb-hyperband/lstm-imdb-hyperband.pth")

In [11]:
def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    print("Accuracy:", correct/total)

In [12]:
test(best_model, test_dl, epochs)

/ext3/miniconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:691: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:925.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Accuracy: 0.8694
